<a href="https://colab.research.google.com/github/Meghna1904/compilerlab/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [2]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bison is already the newest version (2:3.8.2+dfsg-1build1).
flex is already the newest version (2.6.4-8build2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


## Lex only

In [10]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Writing program.l


if you want to use at txt as an input

In [11]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [12]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [25]:
#@title Writing YACC program
%%writefile program.y

%{
    #include<stdio.h>
    #include<stdlib.h>
%}
%token DIGIT LETTER UND NL
%%
stmt: variable NL {printf("Valid Identifier\n");exit(0);}
variable: LETTER alphanumeric;
alphanumeric: LETTER alphanumeric
            | DIGIT alphanumeric
            | UND alphanumeric
            | LETTER
            | DIGIT
            | UND;
%%

int yyerror(){
    printf("Invalid Identifier\n");
    exit(0);
}

void main(){
    printf("Enter the variable name: ");
    yyparse();
}

Overwriting program.y


In [26]:
#@title Writing Lex program
%%writefile program.l

%{
    #include "y.tab.h"
%}
%%
[a-zA-Z] {return LETTER;}
[0-9] {return DIGIT;}
[_] {return UND;}
\n {return NL;}
. {return yytext[0];}
%%

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

In [ ]:
m

In [27]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1024:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1024 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1165:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1165 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter the variable name: name
Valid Identifier


In [13]:
%%writefile program1.l
%{
    #include "y.tab.h"
%}
%%
"a"     { return A; }
"b"     { return B; }
\n      { return NL; }
.       { return yytext[0]; }
%%
int yywrap() { return 1; }

Overwriting program1.l


In [14]:
%%writefile program1.y

%{
#include <stdio.h>
#include <stdlib.h>
int yylex(void);
void yyerror(char *s);
%}
%token A B NL
%%
stmt: S NL { printf("Valid string\n"); exit(0); };
S   : A S | B S | A B B;
%%
void yyerror(char *s) { printf("Invalid string\n"); exit(0); }
int main() { printf("Enter string: "); yyparse(); return 0; }

Writing program1.y


In [15]:
%%shell
yacc -d program1.y
lex program1.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter string: abb
Valid string


In [38]:
%%writefile program2.l

%{
  #include <stdio.h>
  #include <stdlib.h>
%}

%%
"//"(.*)  {}
"/*"(.|\n)*?"*/" {}
. {ECHO;}
%%

int yywrap() {return 1;}

int main() {
  yylex();
}

Overwriting program2.l


In [30]:
%%writefile program2.c
int main(){
  //This is program
  printf("My name is Meh");
  /*helo
  hy*/
}

Overwriting program2.c


In [63]:
%%writefile count.l
%{
#include <stdio.h>
#include <stdlib.h>
int chars=0,lines=0,words=0,sent=0;
%}

%%
[.!?] {sent++;}
[^ \t\n]+ {words++; chars+=yyleng;}
\n {lines++;}
. {chars++;}
%%

int yywrap(){return 1;}

int main(){
    yylex();
    printf("Count of words : %d", words );
    printf("Count of lines : %d", lines );
    printf("Count of character : %d", chars );
    printf("Count of sentences : %d", sent );
}

Overwriting count.l


In [6]:
%%writefile count.txt
meghna

Overwriting count.txt


In [69]:
%%shell

lex count.l
gcc lex.yy.c
./a.out < count.txt

Count of words : 6Count of lines : 3Count of character : 23Count of sentences : 1

LEX PRACTICE




> Add blockquote





```
# This is formatted as code
```

# Lex program


In [3]:
%%writefile vowel.l
%{
#include <stdio.h>
int vowel=0,cons=0;
%}

%%
[aeiouAEIOU] {vowel++;}
[a-zA-Z] {cons++;}
%%

int yywrap(){return 1;}

int main(){
  yylex();
  printf("Vowel Count = : %d\n",vowel);
  printf("Consonants Count = : %d\n",cons);
}

Writing vowel.l


In [9]:
%%shell

lex vowel.l
gcc lex.yy.c
./a.out < count.txt


Vowel Count = : 2
Consonants Count = : 4


frequency of characters


In [8]:
%%writefile frequency.l
%{
    #include <stdio.h>
    int freq[256] ={0};
%}

%%
. {freq[yytext[0]]++;}
\n {}
%%

int yywrap(){return 1;}
int main(){
  yylex();
  for(int i=0;i<256;i++){
    if(freq[i]>0){
      printf("%c : %d\n",i,freq[i]);
    }

  }
  return 0;
}

Overwriting frequency.l


In [9]:
%%shell
lex frequency.l
gcc lex.yy.c
./a.out < count.txt

a : 1
e : 1
g : 1
h : 1
m : 1
n : 1


Convert substring abc to ABC

In [10]:
%%writefile substring.l
%{
    #include <stdio.h>

%}

%%
"abc" {printf("ABC");}
.|\n  {ECHO;}
%%

int yywrap(){return 1;}

int main(){
  printf("Enter the string ");
  yylex();
  return 0;

}

Writing substring.l


In [12]:
%%shell
lex substring.l
gcc lex.yy.c
./a.out

Enter the string abc is happy.
ABC is happy.


CalledProcessError: Command 'lex substring.l
gcc lex.yy.c
./a.out
' died with <Signals.SIGINT: 2>.